# Corrected LSTM

My previous version was doing the recursion incorrectly, so this is me checking with a corrected version of recursion and using all the features and then I'll go build the class a la Marcos and do the grid search to tune parameters. I'm not going to be super good about using a pipeline here, but I put it in the header anyway

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import sys
sys.path.insert(0, '../')
from gw_tools.gw_plot import gw_plot
from gw_tools import data_prep

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import load_model

2023-11-20 12:46:50.055998: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 12:46:50.089933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 12:46:50.089955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 12:46:50.089986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 12:46:50.096706: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 12:46:50.097120: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [33]:
WINDOW_SIZE = 6 # this will be a tuneable parameter later

def to_X_y_multi(scaled_np, window_size=6):
    X = []
    y = []
    for i in range(len(scaled_np)-window_size):
        row = [a for a in scaled_np[i:i+window_size]]
        X.append(row)
        label = scaled_np[i+window_size][0]
        y.append(label)
    return np.array(X), np.array(y)

## Prepare the data
signals and scaling

In [34]:
df = data_prep.select_features(pd.read_pickle('../data/pickled_data/AFL259_short.pkl'))
df = data_prep.add_toy_signal(df).set_index('date')

df.head(3)

,avg_well_depth,gage_ht,discharge_cfs,prcp,temp_avg,hum_avg,hPa_avg,wind_avg,gust_avg,prcp_lag_45D,year_sin,year_cos
date,,,,,,,,,,,,
2005-08-21,99.059167,17.02,600.0,0.0,75.903750,44.166667,1015.333333,7.492083,0.000000,0.00,-0.756711,-0.653749
2005-08-22,99.163333,17.02,599.0,0.0,72.854167,43.041667,1012.458333,10.093333,0.717500,0.16,-0.767845,-0.640635
2005-08-23,99.155000,17.02,598.0,0.0,65.272083,46.041667,1010.000000,14.121250,6.141667,0.72,-0.778752,-0.627332


In [35]:
# including the target (since past target data is a feature)
NUM_FEATS = len(df.columns)

In [36]:
TEST_SIZE = 365 # size of the test set

df_train = df[:-2*TEST_SIZE].copy()
df_val = df[-2*TEST_SIZE:-TEST_SIZE].copy()
df_test = df[-TEST_SIZE:].copy()

y_train = df_train.avg_well_depth.copy().values
y_val = df_val.avg_well_depth.copy().values
y_test = df_test.avg_well_depth.copy().values

In [38]:
df_train.shape, y_train.shape

((3583, 12), (3583,))

In [5]:
scalar = StandardScaler()

scalar.fit(df_train)

/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


StandardScaler()

In [6]:
df_s_train = scalar.transform(df_train)
df_s_val = scalar.transform(df_val)
df_s_test = scalar.transform(df_test)
# these are numpy arrays with the same layout as the df

/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

## Build and Train the Model

In [18]:
LSTM_mod = Sequential()
LSTM_mod.add(InputLayer((WINDOW_SIZE, NUM_FEATS)))
LSTM_mod.add(LSTM(64))
LSTM_mod.add(Flatten())
LSTM_mod.add(Dense(8, 'relu'))
LSTM_mod.add(Dense(1, 'linear'))

LSTM_mod.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                19712     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 20241 (79.07 KB)
Trainable params: 20241 (79.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
cp = ModelCheckpoint('mereLSTM/', save_best_only=True)
# the above makes a folder that our gitignore should exclude

LSTM_mod.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

### reshape training and val data

In [29]:
X_train, y_train = to_X_y_multi(df_s_train)
X_val, y_train = to_X_y_multi(df_s_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((3577, 6, 12), (359,), (359, 6, 12), (365,))

In [32]:
X,y = to_X_y_multi(df_train.to_numpy())

X.shape, y.shape

((3577, 6, 12), (3577,))

In [27]:
LSTM_mod.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, callbacks=[cp])

ValueError: Data cardinality is ambiguous:
  x sizes: 3577
  y sizes: 359
Make sure all arrays contain the same number of samples.

## Testing the Model

In [ ]:
def plot_predictions2(predictions, y):
    #predictions = model.predict(X).flatten()
    # now it just plots
    df_plot = pd.DataFrame(data={'Predictions':predictions, 'Actuals':y})
    
    Pred = df_plot.Predictions.values*-1
    Act = df_plot.Actuals.values*-1
    
    fig, ax1 = plt.subplots(figsize=(16, 5)) 
    plt.suptitle('Prediction', fontsize=16)

    ax1 = plt.subplot()
    plt.plot(Pred, 'r--', label = 'Predicted Water Level')
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Depth from Land Surface in feet', fontsize=14)

    ax2 = plt.subplot()
    ax2.plot(Act, 'b-', label = 'Actual Water Level')

    fig.legend()

    fig.tight_layout()
    plt.show()
    
    return df_plot, mse(y, predictions)

In [ ]:
fill_preds = pd.DataFrame({'Predictions':df_train.avg_well_depth[-WINDOW_SIZE:]})

actual_test = df[-TEST_SIZE-WINDOW_SIZE:].copy().merge(fill_preds, how='outer', on='date')
actual_test.drop(columns=['avg_well_depth'], inplace=True)
actual_test.head(10)